# Analiza N-gramów

Celem tych laboratoriów jest zgłębienie koncepcji n-gramów poprzez implementację własnych rozwiązań i eksperymentowanie z danymi, a następnie wykorzystanie biblioteki NLTK do porównania rezultatów.

---

## Część I: Implementacja Ręczna


### Opis
- **N-gramy** to sekwencje kolejnych elementów (np. słów, liter) wyodrębnianych z tekstu.
- Przykładowo, dla zdania `"kot i pies"` oraz n=2 (bigramy) spodziewamy się uzyskać:  
  `["kot i", "i pies"]`.

# Model n-gramów w Pythonie

Poniższy kod umożliwia:
- Wczytanie tekstu z pliku lub użycie domyślnego tekstu.
- Budowanie modelu n-gramów dla zadanej wartości n (dopuszczalne: 2, 3, 4, 8, 16).
- Wyświetlenie przykładowych n-gramów z listą słów, które następują po nich.
- Generowanie tekstu na podstawie wytrenowanego modelu.

## Kod źródłowy

```python
import random
import sys
import argparse

def build_ngram_model(text, n):
    """
    Buduje model n-gramów na podstawie zadanego tekstu.
    
    :param text: Ciąg tekstowy wejściowy.
    :param n: Liczba tokenów tworzących n-gram.
    :return: Słownik, którego klucze to krotki zawierające n kolejnych tokenów,
             a wartości to lista słów występujących po danym n-gramie.
    """
    tokens = text.split()
    model = {}
    for i in range(len(tokens) - n):
        gram = tuple(tokens[i:i+n])
        next_word = tokens[i+n]
        if gram not in model:
            model[gram] = []
        model[gram].append(next_word)
    return model

def generate_text(model, n, length, seed=None):
    """
    Generuje tekst na podstawie modelu n-gramów.
    
    :param model: Model n-gramów w postaci słownika.
    :param n: Liczba tokenów w n-gramie.
    :param length: Liczba dodatkowych słów do wygenerowania.
    :param seed: Opcjonalny początkowy n-gram (krotka). Jeżeli nie podany, wybierany jest losowo.
    :return: Wygenerowany ciąg tekstowy.
    """
    if seed is None:
        seed = random.choice(list(model.keys()))
    output = list(seed)
    current = seed
    for _ in range(length):
        possibilities = model.get(current)
        if not possibilities:
            break  # brak kontynuacji - kończymy generowanie
        next_word = random.choice(possibilities)
        output.append(next_word)
        current = tuple(output[-n:])
    return ' '.join(output)

def main():
    parser = argparse.ArgumentParser(
        description="Generowanie tekstu na podstawie modelu n-gramów. "
                    "Wybierz wartość n oraz opcjonalnie podaj plik z tekstem."
    )
    parser.add_argument('--n', type=int, default=2,
                        help="Wartość n dla n-gramów (dopuszczalne: 2, 3, 4, 8, 16)")
    parser.add_argument('--length', type=int, default=20,
                        help="Liczba dodatkowych słów do wygenerowania (domyślnie 20)")
    parser.add_argument('--file', type=str,
                        help="Ścieżka do pliku tekstowego (jeżeli nie podano, używany jest przykładowy tekst)")
    args = parser.parse_args()

    # Wczytywanie tekstu z pliku lub użycie tekstu przykładowego
    if args.file:
        try:
            with open(args.file, 'r', encoding='utf-8') as f:
                text = f.read()
        except Exception as e:
            print(f"Błąd przy otwieraniu pliku: {e}")
            sys.exit(1)
    else:
        text = (
            "To jest przykładowy tekst do demonstracji implementacji modelu n-gramów. "
            "Model n-gramów pozwala analizować sekwencje słów i generować nowe teksty na podstawie wzorców. "
            "Wybierz odpowiednią wartość n, aby zobaczyć, jak zmienia się generowany tekst. "
            "Im większa wartość n, tym model bierze pod uwagę dłuższy kontekst, co może wpływać na spójność i precyzję generowanego tekstu. "
            "Eksperymentuj z różnymi ustawieniami, np. 2, 3, 4, 8, 16-gramów."
        )
    # Usunięcie zbędnych znaków nowej linii (jeśli występują)
    text = text.replace('\n', ' ')

    # Sprawdzamy, czy wybrana wartość n jest dopuszczalna
    if args.n not in [2, 3, 4, 8, 16]:
        print("Proszę wybrać wartość n spośród: 2, 3, 4, 8, 16.")
        sys.exit(1)
    
    n = args.n
    print(f"\nBudowanie modelu {n}-gramów...")
    model = build_ngram_model(text, n)

    # Wyświetlenie kilku przykładowych n-gramów z listą możliwych kontynuacji
    print("\nPrzykładowe n-gramy i słowa, które następują po nich:")
    for idx, (gram, next_words) in enumerate(model.items()):
        print(f"{gram} -> {next_words}")
        if idx >= 4:
            break

    # Generowanie tekstu na podstawie modelu n-gramów
    print("\nGenerowanie tekstu:")
    generated_text = generate_text(model, n, args.length)
    print(generated_text)




# Implementacja Laplace Smoothing dla modelu bigramów

Laplace smoothing (wygładzanie add-one) to metoda pozwalająca na uniknięcie zerowych prawdopodobieństw w modelach językowych, szczególnie przy modelach n-gramowych. Polega ona na dodaniu 1 do liczby wystąpień każdego n-gramu, dzięki czemu nawet nieobserwowane sekwencje otrzymują niezerową wartość prawdopodobieństwa.

## Matematyka
Dla danego kontekstu \(h\) oraz kolejnego słowa \(w\), prawdopodobieństwo oblicza się według wzoru:

$$
P(w \mid h) = \frac{\mathrm{count}(h, w) + 1}{\mathrm{count}(h) + V}
$$

gdzie:
- count(h, w) – liczba wystąpień sekwencji \(h, w\),
- count(h) – liczba wystąpień kontekstu \(h\),
- (V) – rozmiar słownika (liczba unikalnych słów).


## Przykładowa implementacja w Pythonie

```python
import random

def build_bigram_counts(text):
    """
    Buduje liczniki bigramów na podstawie tekstu.
    
    :param text: Ciąg tekstowy.
    :return: Słownik, w którym kluczem jest słowo, a wartością inny słownik z licznikami słów występujących po danym słowie.
    """
    tokens = text.split()
    counts = {}
    for i in range(len(tokens) - 1):
        word = tokens[i]
        next_word = tokens[i + 1]
        if word not in counts:
            counts[word] = {}
        counts[word][next_word] = counts[word].get(next_word, 0) + 1
    return counts

def laplace_probability(counts, word, next_word, vocab_size):
    """
    Oblicza prawdopodobieństwo wystąpienia 'next_word' po 'word' z wykorzystaniem Laplace smoothing.
    
    :param counts: Słownik liczników bigramów.
    :param word: Słowo-historyczne.
    :param next_word: Słowo, dla którego liczymy prawdopodobieństwo.
    :param vocab_size: Rozmiar słownika (liczba unikalnych słów w korpusie).
    :return: Wygładzona wartość prawdopodobieństwa.
    """
    count_bigram = counts.get(word, {}).get(next_word, 0)
    total_count = sum(counts.get(word, {}).values())
    return (count_bigram + 1) / (total_count + vocab_size)

# Przykładowy tekst
text = (
    "To jest przykładowy tekst do demonstracji modelu bigramów. "
    "Model bigramów jest prostym modelem językowym."
)

# Budujemy liczniki bigramów
bigram_counts = build_bigram_counts(text)
print("Bigram counts:")
for word, next_words in bigram_counts.items():
    print(f"{word}: {next_words}")

# Przygotowanie słownika - zestaw unikalnych słów
vocab = set(text.split())
vocab_size = len(vocab)
print(f"\nRozmiar słownika: {vocab_size}")

# Obliczenie prawdopodobieństwa dla konkretnego bigramu z Laplace smoothing
word = "bigramów"
next_word = "jest"
prob = laplace_probability(bigram_counts, word, next_word, vocab_size)
print(f"\nPrawdopodobieństwo wystąpienia słowa '{next_word}' po '{word}': {prob:.4f}")

# Przykładowe obliczenie dla nieobserwowanego bigramu
unknown_next = "nieistnieje"
prob_unknown = laplace_probability(bigram_counts, word, unknown_next, vocab_size)
print(f"Prawdopodobieństwo wystąpienia słowa '{unknown_next}' po '{word}': {prob_unknown:.4f}")


## Część II: Wykorzystanie NLTK w N-grams


1. **Tokenizacja i Generacja n-gramów:**
   - Użyj funkcji `nltk.word_tokenize` do podziału tekstu na słowa.
   - Wykorzystaj funkcje takie jak `nltk.ngrams`, `nltk.bigrams` lub `nltk.trigrams` do wygenerowania n-gramów.

2. **Budowa Modelu Językowego:**
   - Wybierz korpus dostępny w NLTK (np. `reuters` lub `brown`).
   - Tokenizuj teksty z wybranego korpusu.
   - Wygeneruj trigrams (lub n-gramy o wybranej długości).
   - Zbuduj model, który zapisze częstotliwość występowania kolejnych słów po danej sekwencji (np. dla pary słów).
   - Znormalizuj liczniki do postaci prawdopodobieństw.

### Przykładowy kod:
```python
import nltk
from nltk import trigrams
from nltk.corpus import reuters
from collections import defaultdict

# Pobranie niezbędnych zasobów
nltk.download('reuters')
nltk.download('punkt')

# Tokenizacja tekstu z korpusu Reuters
words = nltk.word_tokenize(' '.join(reuters.words()))

# Generowanie trigrams
tri_grams = list(trigrams(words))

# Budowa modelu trigramowego
model = defaultdict(lambda: defaultdict(lambda: 0))
for w1, w2, w3 in tri_grams:
    model[(w1, w2)][w3] += 1

# Normalizacja do prawdopodobieństw
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

def predict_next_word(w1, w2):
    """
    Przewiduje kolejne słowo na podstawie dwóch poprzednich słów przy użyciu modelu trigramowego.
    """
    next_words = model[(w1, w2)]
    if next_words:
        return max(next_words, key=next_words.get)
    else:
        return "Brak predykcji"

# Przykład użycia funkcji predykcji
print("Next Word:", predict_next_word('the', 'stock'))


## Część III: Metryki Modelowania Języka


Aby ocenić jakość modelu językowego, wykorzystujemy kilka kluczowych metryk:

### 1. Entropia

Entropia mierzy ilość informacji zawartej w rozkładzie prawdopodobieństwa. Wzór:
  
$$
H(p) = \sum_{x} p(x) \cdot \left(-\log(p(x))\right)
$$

> **Ważne:** Wartość entropii \( H(p) \) zawsze jest większa lub równa 0.

---

### 2. Cross-Entropy

Cross-Entropy (krzyżowa entropia) ocenia, jak dobrze model przewiduje dane testowe. Dla ciągu słów \( w_1, w_2, \dots, w_N \) wzór jest następujący:

$$
H(p) = \sum_{i=1}^{N} \left(-\log_2 \left(p(w_i \mid w_1^{i-1})\right)\right)
$$

> **Uwaga:** Cross-Entropy jest zawsze większa lub równa entropii – model nie może być bardziej pewny niż prawdziwa niepewność danych.

---

### 3. Perplexity

Perplexity jest miarą niepewności modelu i jakości jego przewidywań. Można ją obliczyć z wykorzystaniem cross-entropy:

$$
\text{Perplexity} = 2^{H(p)}
$$

Alternatywnie, dla zbioru testowego, wzór przyjmuje postać:

$$
\text{PP}(W) = \left( \prod_{i=1}^{N} P(w_i \mid w_{i-1}) \right)^{-\frac{1}{N}}
$$

---

### Przykład Obliczeń

Rozważmy zdanie: **"Natural Language Processing"**.

#### Krok 1: Prawdopodobieństwa

Dla pierwszego słowa (przyjmując `<start>` jako kontekst):

| Słowo      | \( P(\text{word} \mid \text{<start>}) \) |
|------------|-----------------------------------------|
| The        | 0.4                                     |
| Processing | 0.3                                     |
| Natural    | 0.12                                    |
| Language   | 0.18                                    |

Zakładamy, że pierwszym wybranym słowem jest **"Natural"**.

Dla słowa po **"Natural"**:

| Słowo      | \( P(\text{word} \mid \text{Natural}) \) |
|------------|------------------------------------------|
| The        | 0.05                                     |
| Processing | 0.3                                      |
| Natural    | 0.15                                     |
| Language   | 0.5                                      |

Wybieramy **"Language"**.

Dla słowa po **"Language"**:

| Słowo      | \( P(\text{word} \mid \text{Language}) \) |
|------------|------------------------------------------|
| The        | 0.1                                      |
| Processing | 0.7                                      |
| Natural    | 0.1                                      |
| Language   | 0.1                                      |

Wybieramy **"Processing"**.

#### Krok 2: Obliczenie Perplexity

Obliczamy iloczyn prawdopodobieństw:

$$
0.12 \times 0.5 \times 0.7 = 0.042
$$

Perplexity wyliczamy jako:

$$
\text{PP}(W) = \left(0.042\right)^{-\frac{1}{3}} \approx 2.876
$$

#### Krok 3: Obliczenie Entropii

Entropię uzyskujemy jako logarytm dwójkowy z wartości perplexity:

$$
\text{Entropy} = \log_2(2.876) \approx 1.524
$$

---

### Podsumowanie

- **Entropia:** Mierzy ilość informacji zawartej w rozkładzie prawdopodobieństwa.
- **Cross-Entropy:** Ocena, jak dobrze model przewiduje dane testowe – zawsze większa lub równa entropii.
- **Perplexity:** Miara niepewności modelu; im niższa wartość, tym lepsze przewidywania.



Dodatkowo laboratoria mają na celu zapoznanie z metodami Bag of Words, TF-IDF i embeddingów.

# Metody reprezentacji tekstu


---

Bag-of-Words (BoW) to jedna z najprostszych metod reprezentacji tekstu, która polega na zamianie dokumentu na wektor liczbowy. W tej metodzie tekst traktowany jest jako "worek" słów, gdzie istotna jest jedynie liczba wystąpień poszczególnych tokenów, a kolejność słów zostaje zignorowana.

**Proces przetwarzania:**
- **Tokenizacja:** Podział tekstu na mniejsze jednostki (słowa, tokeny).
- **Czyszczenie:** Usuwanie znaków interpunkcyjnych, liczb oraz konwersja tekstu do formy jednolitej (np. na małe litery).
- **Usuwanie stop-słów:** Eliminacja często występujących, ale mało informacyjnych słów (np. „i”, „oraz”, „ale”).
- **Budowanie słownika:** Utworzenie listy unikalnych słów, które występują w korpusie.
- **Tworzenie wektora:** Reprezentacja dokumentu jako wektor częstotliwości występowania poszczególnych słów.

**Biblioteki:**
- **scikit-learn:** Narzędzie `CountVectorizer` umożliwia szybkie przekształcenie tekstu na macierz cech.
- **NLTK:** Umożliwia tokenizację, usuwanie stop-słów, stemizację i lematyzację.
- **spaCy:** Zaawansowane narzędzie NLP, które również pozwala na tokenizację oraz inne operacje przetwarzania języka.

**Zalety i wady:**
- **Zalety:** Prosta implementacja, efektywna przy analizie częstotliwości słów.
- **Wady:** Utrata informacji o kolejności słów i kontekście, co może ograniczać zdolność uchwycenia semantyki dokumentu.


# Przykład 1: Bag-of-Words (BoW) przy użyciu scikit-learn

```python
from sklearn.feature_extraction.text import CountVectorizer

# Przykładowy korpus dokumentów
documents = [
    "To jest przykładowy tekst.",
    "Drugi dokument z innym tekstem.",
    "Trzeci dokument zawiera przykładowy tekst."
]

# Inicjalizacja CountVectorizer z polskimi stop-słowami
vectorizer = CountVectorizer(stop_words='polish')
bow_matrix = vectorizer.fit_transform(documents)

# Wyświetlenie słownika (wszystkich tokenów)
print("Słownik tokenów:")
print(vectorizer.get_feature_names_out())

# Wyświetlenie macierzy Bag-of-Words
print("\nMacierz Bag-of-Words:")
print(bow_matrix.toarray())


# TF-IDF (Term Frequency – Inverse Document Frequency)


TF-IDF to technika oceny ważności słowa w dokumencie, która bierze pod uwagę zarówno jego częstotliwość występowania w danym dokumencie (TF), jak i rzadkość występowania w całym zbiorze dokumentów (IDF). Dzięki temu słowa charakterystyczne dla danego dokumentu są wyróżniane, a te powszechnie występujące zyskują mniejszą wagę.

**Proces przetwarzania:**
- **Obliczanie TF (Term Frequency):** Mierzy, jak często dane słowo pojawia się w danym dokumencie.
- **Obliczanie IDF (Inverse Document Frequency):** Mierzy, jak unikalne jest dane słowo w całym zbiorze dokumentów, zazwyczaj przy użyciu logarytmu odwrotnej proporcji liczby dokumentów zawierających dane słowo.
- **Kombinacja:** Mnożenie TF przez IDF daje wynikową wagę, która wskazuje na znaczenie słowa w kontekście dokumentu i korpusu.

**Biblioteki:**
- **scikit-learn:** `TfidfVectorizer` umożliwia automatyczne obliczanie macierzy TF-IDF na podstawie zbioru dokumentów.
- **NLTK:** Może być wykorzystywany do wstępnej obróbki tekstu (tokenizacja, usuwanie stop-słów) przed obliczeniem TF-IDF.
- **spaCy:** Ułatwia przygotowanie danych poprzez zaawansowaną analizę składniową i tokenizację.

**Zalety i wady:**
- **Zalety:** Lepsze od BoW przy wyłapywaniu istotnych słów, ponieważ uwzględnia unikalność słów w korpusie.
- **Wady:** W przypadku bardzo dużych zbiorów danych obliczenia mogą być kosztowne, a metoda nie przechwytuje semantycznych zależności między słowami.



```markdown
# Przykład 2: TF-IDF przy użyciu scikit-learn

```python
from sklearn.feature_extraction.text import TfidfVectorizer

# Przykładowy korpus dokumentów
documents = [
    "To jest przykładowy tekst.",
    "Drugi dokument z innym tekstem.",
    "Trzeci dokument zawiera przykładowy tekst."
]

# Inicjalizacja TfidfVectorizer z polskimi stop-słowami
tfidf_vectorizer = TfidfVectorizer(stop_words='polish')
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Wyświetlenie słownika (wszystkich tokenów)
print("Słownik tokenów (TF-IDF):")
print(tfidf_vectorizer.get_feature_names_out())

# Wyświetlenie macierzy TF-IDF
print("\nMacierz TF-IDF:")
print(tfidf_matrix.toarray())


# Word Embeddings

Word Embeddings to metoda reprezentacji słów, która mapuje każde słowo na wektor w przestrzeni wielowymiarowej. Dzięki temu słowa o podobnym znaczeniu mają wektory umieszczone blisko siebie, co pozwala na uchwycenie zależności semantycznych i syntaktycznych między nimi. Modele te uczą się reprezentacji na podstawie kontekstu, w jakim słowa występują w dużych zbiorach tekstowych.

**Popularne modele:**
- **Word2Vec:** Używa architektur CBOW (Continuous Bag of Words) lub Skip-Gram do nauki wektorowych reprezentacji słów na podstawie ich kontekstu.
- **GloVe:** Wykorzystuje statystyki współwystępowania słów w korpusie, aby uzyskać globalną reprezentację wektorową.
- **FastText:** Rozszerzenie Word2Vec, które uwzględnia również wewnętrzną strukturę słów, dzieląc je na mniejsze fragmenty (n-gramy).

**Biblioteki:**
- **gensim:** Bardzo popularna biblioteka do trenowania modeli Word2Vec, FastText oraz korzystania z gotowych modeli embeddingowych.
- **TensorFlow/Keras:** Umożliwiają budowanie niestandardowych modeli sieci neuronowych, w tym warstw embeddingowych, co pozwala na głębszą integrację z zadaniami uczenia maszynowego.
- **PyTorch:** Alternatywna platforma do trenowania modeli sieci neuronowych, oferująca elastyczność w implementacji modeli embeddingowych.

**Zalety i wady:**
- **Zalety:** Pozwalają na uchwycenie głębszych relacji semantycznych między słowami, co znacząco poprawia wydajność w zadaniach takich jak klasyfikacja tekstu, analiza sentymentu czy tłumaczenie maszynowe.
- **Wady:** Wymagają dużych zbiorów danych do skutecznego treningu, a ich trenowanie bywa czasochłonne i wymaga znacznych zasobów obliczeniowych.



```markdown
# Przykład 3: Word Embeddings przy użyciu gensim (Word2Vec)

```python
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Przygotowanie przykładowego korpusu - tokenizacja tekstów
sentences = [
    simple_preprocess("To jest przykładowy tekst."),
    simple_preprocess("Drugi dokument z innym tekstem."),
    simple_preprocess("Trzeci dokument zawiera przykładowy tekst.")
]

# Trenowanie modelu Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Wybór słowa do analizy
word = "tekst"

# Sprawdzenie, czy słowo istnieje w modelu oraz wyświetlenie jego wektora
if word in model.wv:
    print(f"Wektor dla słowa '{word}':")
    print(model.wv[word])
else:
    print(f"Słowo '{word}' nie występuje w modelu.")

# Znalezienie 3 najbliższych (podobnych) słów
similar_words = model.wv.most_similar(word, topn=3)
print(f"\nNajbardziej podobne słowa do '{word}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity:.4f}")


# Zadanie

# Zadania do zrobienia: Reprezentacja Tekstu i Analiza N-gramów

[Dataset 20newsgroups](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)

```python
from sklearn.datasets import fetch_20newsgroups

# Wybieramy kilka kategorii do analizy
categories = ['sci.med', 'sci.space', 'rec.sport.baseball', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

print(f"Pobrano {len(documents)} dokumentów.")



# Zadanie 1: Reprezentacja tekstu przy użyciu metody Bag-of-Words (BoW)

**Polecenie:**
- Wczytaj dokumenty z wbudowanego datasetu (np. 20 Newsgroups).
- Wykonaj tokenizację tekstu oraz usuń stop-słowa.
- Zbuduj reprezentację tekstu metodą Bag-of-Words, tworząc macierz wystąpień słów.
- Wyświetl najczęściej występujące słowa wraz z ich liczebnością.

---

# Zadanie 2: Obliczanie TF-IDF dla zbioru dokumentów

**Polecenie:**
- Wczytaj dokumenty z wbudowanego datasetu.
- Przetwórz tekst, wykonując tokenizację oraz usuwanie stop-słów.
- Oblicz macierz TF-IDF dla całego zbioru dokumentów.
- Dla wybranego dokumentu wypisz słowa o najwyższych wartościach TF-IDF.

---

# Zadanie 3: Implementacja Word Embeddings

**Polecenie:**
- Wczytaj i przetwórz dokumenty z wbudowanego datasetu (tokenizacja, usuwanie stop-słów, normalizacja).
- Wytrenuj model Word Embeddings (np. Word2Vec lub GloVe) na przetworzonym korpusie.
- Przetestuj model, wyszukując najbliższe wektory (sąsiadów) dla wybranego słowa.

---

# Zadanie 4: Generowanie i analiza Bigramów

**Polecenie:**
- Wczytaj dokumenty z wbudowanego datasetu.
- Wykonaj tokenizację tekstu oraz usuń stop-słowa.
- Wygeneruj bigramy (pary kolejnych słów) z przetworzonego tekstu.
- Wyświetl najczęściej występujące bigramy wraz z ich liczebnością.

---

# Zadanie 5: Analiza Trigramów w tekście

**Polecenie:**
- Wczytaj dokumenty z wbudowanego datasetu.
- Przetwórz tekst, wykonując tokenizację oraz usuwanie stop-słów.
- Wygeneruj trigramy (sekwencje trzech kolejnych słów) z dokumentów.
- Wypisz najczęściej występujące trigramy w analizowanym zbiorze.


Więcej informacji:

- [Word2vec](https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/
)

- [TFIDF](https://www.geeksforgeeks.org/understanding-tf-idf-term-frequency-inverse-document-frequency/
)

- [N-grams NLTK](https://www.geeksforgeeks.org/n-gram-language-modelling-with-nltk/)

- [NLTK](https://www.geeksforgeeks.org/tokenize-text-using-nltk-python/)